# 라이브러리 로딩

In [1]:
import numpy as np # Numpy
import pandas as pd # Pandas
import matplotlib as mpl #Matplotlib 세팅용
import matplotlib.pyplot as plt # 시각화 도구
import seaborn as sns # 시각화 도구
from sklearn.model_selection import train_test_split # 데이터셋 분리
from sklearn.model_selection import KFold # KFold 교차검증
from sklearn.cluster import KMeans # 클러스터링
from sklearn.metrics import silhouette_score # 실루엣 점수
import xgboost as xgb # XGBoost
from sklearn.model_selection import GridSearchCV # 그리드 서치
from sklearn.metrics import accuracy_score, precision_score # 평가 지표
from sklearn.metrics import recall_score, confusion_matrix, roc_auc_score, f1_score # 평가 지표
from imblearn.combine import SMOTEENN, SMOTETomek # 복합샘플링
from hyperopt import hp, fmin, tpe, Trials # HyperOPT

import warnings # 경고문 제거용


%matplotlib inline
%config Inlinebackend.figure_format = 'retina'

# 한글 폰트 설정
mpl.rc('font', family='D2Coding')
# 유니코드에서 음수 부호 설정
mpl.rc('axes', unicode_minus = False)

warnings.filterwarnings('ignore')
sns.set(font="D2Coding", rc={"axes.unicode_minus":False}, style='darkgrid')
plt.rc('figure', figsize=(10,8))

# 데이터 불러오기

In [2]:
data = pd.read_excel('train_test_na_filled.xlsx', sheet_name='Train')

# 전처리

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 18 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   8693 non-null   object 
 1   HomePlanet    8693 non-null   object 
 2   CryoSleep     8693 non-null   bool   
 3   Cabin1        8590 non-null   object 
 4   Cabin2        8590 non-null   float64
 5   Combi         8590 non-null   object 
 6   Cabin3        8590 non-null   object 
 7   Cabin         8590 non-null   object 
 8   Destination   8693 non-null   object 
 9   Age           8693 non-null   int64  
 10  VIP           8693 non-null   bool   
 11  RoomService   8693 non-null   int64  
 12  FoodCourt     8693 non-null   int64  
 13  ShoppingMall  8693 non-null   int64  
 14  Spa           8693 non-null   int64  
 15  VRDeck        8693 non-null   int64  
 16  Name          8493 non-null   object 
 17  Transported   8693 non-null   bool   
dtypes: bool(3), float64(1), int6

## 필요없는 features 제거

In [4]:
# 필요없는 features 제거
data.drop(['PassengerId', 'Cabin', 'Cabin2', 'Name', 'Combi'], axis=1, inplace=True)

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   HomePlanet    8693 non-null   object
 1   CryoSleep     8693 non-null   bool  
 2   Cabin1        8590 non-null   object
 3   Cabin3        8590 non-null   object
 4   Destination   8693 non-null   object
 5   Age           8693 non-null   int64 
 6   VIP           8693 non-null   bool  
 7   RoomService   8693 non-null   int64 
 8   FoodCourt     8693 non-null   int64 
 9   ShoppingMall  8693 non-null   int64 
 10  Spa           8693 non-null   int64 
 11  VRDeck        8693 non-null   int64 
 12  Transported   8693 non-null   bool  
dtypes: bool(3), int64(6), object(4)
memory usage: 704.7+ KB


## 처리하기 힘든 결측값 제거

In [6]:
data.isna().sum()

HomePlanet        0
CryoSleep         0
Cabin1          103
Cabin3          103
Destination       0
Age               0
VIP               0
RoomService       0
FoodCourt         0
ShoppingMall      0
Spa               0
VRDeck            0
Transported       0
dtype: int64

In [7]:
# 결측값들 제거(Cabin)
data.dropna(axis=0, inplace=True)

## Boolean 캐스팅

In [8]:
# Cabin3의 값을 변환
data['Cabin3'].replace({'P': True,'S': False}, inplace=True)
data['Cabin3'] = data['Cabin3'].astype(bool)

## 원핫인코딩

In [9]:
# 원핫인코딩
train_encoding = pd.get_dummies(data['HomePlanet'])
data=data.drop('HomePlanet',axis=1)
data = data.join(train_encoding)

train_encoding = pd.get_dummies(data['Destination'])
data=data.drop('Destination',axis=1)
data = data.join(train_encoding)

train_encoding = pd.get_dummies(data['Cabin1'])
data=data.drop('Cabin1',axis=1)
data = data.join(train_encoding)

In [10]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8590 entries, 0 to 8692
Data columns (total 24 columns):
 #   Column         Non-Null Count  Dtype
---  ------         --------------  -----
 0   CryoSleep      8590 non-null   bool 
 1   Cabin3         8590 non-null   bool 
 2   Age            8590 non-null   int64
 3   VIP            8590 non-null   bool 
 4   RoomService    8590 non-null   int64
 5   FoodCourt      8590 non-null   int64
 6   ShoppingMall   8590 non-null   int64
 7   Spa            8590 non-null   int64
 8   VRDeck         8590 non-null   int64
 9   Transported    8590 non-null   bool 
 10  Earth          8590 non-null   uint8
 11  Europa         8590 non-null   uint8
 12  Mars           8590 non-null   uint8
 13  55 Cancri e    8590 non-null   uint8
 14  PSO J318.5-22  8590 non-null   uint8
 15  TRAPPIST-1e    8590 non-null   uint8
 16  A              8590 non-null   uint8
 17  B              8590 non-null   uint8
 18  C              8590 non-null   uint8
 19  D     

## 스케일링

In [11]:
# 스케일링
col = ['Age', 'RoomService','FoodCourt','ShoppingMall','Spa','VRDeck']
def data_scaled(df, col):
    for i in col:
        data_mean = df[i].mean()
        data_std = df[i].std()
        scaled = (df[i]-data_mean)/data_std
        df[i]=scaled
    return df

In [12]:
data_scaled(data, col)

,CryoSleep,Cabin3,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,...,PSO J318.5-22,TRAPPIST-1e,A,B,C,D,E,F,G,T
0,False,True,0.712274,False,-0.333743,-0.280785,-0.282832,-0.271469,-0.263361,False,...,0,1,0,1,0,0,0,0,0,0
1,False,False,-0.332624,False,-0.168530,-0.275148,-0.241196,0.220460,-0.224520,True,...,0,1,0,0,0,0,0,1,0,0
2,False,False,2.035811,True,-0.268567,1.959032,-0.282832,5.745469,-0.220106,False,...,0,1,1,0,0,0,0,0,0,0
3,False,False,0.294315,False,-0.333743,0.522818,0.335048,2.711463,-0.092988,False,...,0,1,1,0,0,0,0,0,0,0
4,False,False,-0.889902,False,0.125518,-0.236941,-0.031350,0.234796,-0.261596,True,...,0,1,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,False,True,0.851594,True,-0.333743,3.990274,-0.282832,1.200732,-0.198037,False,...,0,0,1,0,0,0,0,0,0,0
8689,True,False,-0.750583,False,-0.333743,-0.280785,-0.282832,-0.271469,-0.263361,False,...,1,0,0,0,0,0,0,0,1,0
8690,False,False,-0.193304,False,-0.333743,-0.280785,2.834877,-0.270573,-0.263361,True,...,0,1,0,0,0,0,0,0,1,0
8691,False,False,0.224655,False,-0.333743,0.376253,-0.282832,0.044835,2.592370,False,...,0,0,0,0,0,0,1,0,0,0


In [13]:
data.columns

Index(['CryoSleep', 'Cabin3', 'Age', 'VIP', 'RoomService', 'FoodCourt',
       'ShoppingMall', 'Spa', 'VRDeck', 'Transported', 'Earth', 'Europa',
       'Mars', '55 Cancri e', 'PSO J318.5-22', 'TRAPPIST-1e', 'A', 'B', 'C',
       'D', 'E', 'F', 'G', 'T'],
      dtype='object')

# 데이터셋 분리

In [14]:
X_train, X_test, y_train, y_test = train_test_split(data.drop(['Transported'], axis=1),
                                                    data.Transported, random_state=109)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train)

# XGBoost

In [15]:
xgb_search_space = {'max_depth': hp.quniform('max_depth', 5, 15, 1),
                   'min_child_weight': hp.quniform('min_child_weight', 1, 20, 1),
                   'colsample_bytree': hp.uniform('colsample_bytree', 0.5, 0.95),
                   'learning_rate': hp.uniform('learning_rate', 0.01, 0.4),
                   'gamma': hp.uniform('gamma', 0, 4)}

In [16]:
# fmin()에서 호출 시 search_space 값으로 XGBClassifier 교차 검증 학습 후 -1 * roc_auc 평균 값을 반환
def bin_objective_func(search_space):
    xgb_clf = xgb.XGBClassifier(n_estimators=100, max_depth=int(search_space['max_depth']),
                            min_child_weight=int(search_space['min_child_weight']),
                            colsample_bytree=search_space['colsample_bytree'],
                            learning_rate=search_space['learning_rate'],
                            gamma=search_space['gamma'])
    
    # 3개 k-fold 방식으로 평가된 roc_auc 지표를 담는 list
    roc_auc_list = []
    
    # 3개 k-fold 방식 적용
    kf = KFold(n_splits=3)
    
    # X_train을 다시 학습과 검증용 데이터로 분리
    for tr_index, val_index in kf.split(X_train):
        # kf.split(X_train)으로 추출된 학습과 검증 index 값으로 학습과 검증 데이터 세트 분리
        X_tr, y_tr = X_train.iloc[tr_index], y_train.iloc[tr_index]
        X_val, y_val = X_train.iloc[val_index], y_train.iloc[val_index]
        
        # early stopping은 30회로 설정하고 추출된 학습과 검증 데이터로 XGBClassifier 학습 수행
        xgb_clf.fit(X_tr, y_tr, early_stopping_rounds=30, eval_metric="auc",
                   eval_set=[(X_tr, y_tr), (X_val, y_val)])
        
        # 1로 예측한 확률값 추출 후 roc auc 계산하고 평균 roc auc 계산을 위해 list에 결과값 담음.
        score = roc_auc_score(y_val, xgb_clf.predict_proba(X_val)[:,1])
        roc_auc_list.append(score)
        
    # 3개 k-fold로 계산된 roc_auc 값의 평균값을 반환하되,
    # HyperOPT는 목적함수의 최솟값을 위한 입력값을 찾으므로 -1을 곱한 뒤 반환
    return -1*np.mean(roc_auc_list)

In [17]:
trials = Trials()

# fmin() 함수를 호출. max_evals 지정된 횟수만큼 반복 후 목적함수의 최솟값을 가지는 최적 입력값 추출
best = fmin(fn=bin_objective_func,
           space=xgb_search_space,
           algo=tpe.suggest,
           max_evals=50, # 최대 반복 횟수를 지정합니다
           trials=trials, rstate=np.random.default_rng(seed=109))

print('best:', best)

[0]	validation_0-auc:0.88859	validation_1-auc:0.86359                                                                  
[1]	validation_0-auc:0.90650	validation_1-auc:0.87578                                                                  
[2]	validation_0-auc:0.91651	validation_1-auc:0.87530                                                                  
[3]	validation_0-auc:0.92452	validation_1-auc:0.87867                                                                  
[4]	validation_0-auc:0.92727	validation_1-auc:0.87963                                                                  
[5]	validation_0-auc:0.93083	validation_1-auc:0.87947                                                                  
[6]	validation_0-auc:0.93305	validation_1-auc:0.88061                                                                  
[7]	validation_0-auc:0.93410	validation_1-auc:0.88287                                                                  
[8]	validation_0-auc:0.93558	validation_

[20]	validation_0-auc:0.94724	validation_1-auc:0.88602                                                                 
[21]	validation_0-auc:0.94724	validation_1-auc:0.88602                                                                 
[22]	validation_0-auc:0.94736	validation_1-auc:0.88589                                                                 
[23]	validation_0-auc:0.94783	validation_1-auc:0.88633                                                                 
[24]	validation_0-auc:0.94864	validation_1-auc:0.88658                                                                 
[25]	validation_0-auc:0.94866	validation_1-auc:0.88669                                                                 
[26]	validation_0-auc:0.94937	validation_1-auc:0.88648                                                                 
[27]	validation_0-auc:0.95041	validation_1-auc:0.88634                                                                 
[28]	validation_0-auc:0.95108	validation

[33]	validation_0-auc:0.95273	validation_1-auc:0.88295                                                                 
[34]	validation_0-auc:0.95273	validation_1-auc:0.88295                                                                 
[35]	validation_0-auc:0.95315	validation_1-auc:0.88250                                                                 
[36]	validation_0-auc:0.95315	validation_1-auc:0.88250                                                                 
[37]	validation_0-auc:0.95315	validation_1-auc:0.88250                                                                 
[38]	validation_0-auc:0.95354	validation_1-auc:0.88253                                                                 
[39]	validation_0-auc:0.95410	validation_1-auc:0.88236                                                                 
[40]	validation_0-auc:0.95446	validation_1-auc:0.88255                                                                 
[41]	validation_0-auc:0.95446	validation

[40]	validation_0-auc:0.92205	validation_1-auc:0.88764                                                                 
[41]	validation_0-auc:0.92244	validation_1-auc:0.88789                                                                 
[42]	validation_0-auc:0.92239	validation_1-auc:0.88783                                                                 
[43]	validation_0-auc:0.92269	validation_1-auc:0.88810                                                                 
[44]	validation_0-auc:0.92278	validation_1-auc:0.88816                                                                 
[45]	validation_0-auc:0.92298	validation_1-auc:0.88827                                                                 
[46]	validation_0-auc:0.92316	validation_1-auc:0.88836                                                                 
[47]	validation_0-auc:0.92308	validation_1-auc:0.88853                                                                 
[48]	validation_0-auc:0.92337	validation

[25]	validation_0-auc:0.91799	validation_1-auc:0.88651                                                                 
[26]	validation_0-auc:0.91829	validation_1-auc:0.88588                                                                 
[27]	validation_0-auc:0.91875	validation_1-auc:0.88669                                                                 
[28]	validation_0-auc:0.91919	validation_1-auc:0.88687                                                                 
[29]	validation_0-auc:0.91977	validation_1-auc:0.88748                                                                 
[30]	validation_0-auc:0.91992	validation_1-auc:0.88734                                                                 
[31]	validation_0-auc:0.92012	validation_1-auc:0.88759                                                                 
[32]	validation_0-auc:0.92069	validation_1-auc:0.88819                                                                 
[33]	validation_0-auc:0.92063	validation

...

[66]	validation_0-auc:0.92808	validation_1-auc:0.89271                                                                 
[67]	validation_0-auc:0.92815	validation_1-auc:0.89281                                                                 
[68]	validation_0-auc:0.92834	validation_1-auc:0.89311                                                                 
[69]	validation_0-auc:0.92834	validation_1-auc:0.89311                                                                 
[70]	validation_0-auc:0.92844	validation_1-auc:0.89307                                                                 
[71]	validation_0-auc:0.92844	validation_1-auc:0.89307                                                                 
[72]	validation_0-auc:0.92851	validation_1-auc:0.89323                                                                 
[73]	validation_0-auc:0.92851	validation_1-auc:0.89323                                                                 
[74]	validation_0-auc:0.92851	validation

[34]	validation_0-auc:0.92122	validation_1-auc:0.88665                                                                 
[35]	validation_0-auc:0.92160	validation_1-auc:0.88663                                                                 
[36]	validation_0-auc:0.92188	validation_1-auc:0.88703                                                                 
[37]	validation_0-auc:0.92244	validation_1-auc:0.88681                                                                 
[38]	validation_0-auc:0.92275	validation_1-auc:0.88719                                                                 
[39]	validation_0-auc:0.92274	validation_1-auc:0.88723                                                                 
[40]	validation_0-auc:0.92274	validation_1-auc:0.88723                                                                 
[41]	validation_0-auc:0.92287	validation_1-auc:0.88712                                                                 
[42]	validation_0-auc:0.92319	validation

[2]	validation_0-auc:0.90257	validation_1-auc:0.87072                                                                  
[3]	validation_0-auc:0.90955	validation_1-auc:0.87497                                                                  
[4]	validation_0-auc:0.91099	validation_1-auc:0.87721                                                                  
[5]	validation_0-auc:0.91330	validation_1-auc:0.87589                                                                  
[6]	validation_0-auc:0.91680	validation_1-auc:0.88165                                                                  
[7]	validation_0-auc:0.91955	validation_1-auc:0.88203                                                                  
[8]	validation_0-auc:0.92112	validation_1-auc:0.88242                                                                  
[9]	validation_0-auc:0.92133	validation_1-auc:0.88393                                                                  
[10]	validation_0-auc:0.92200	validation

[21]	validation_0-auc:0.93669	validation_1-auc:0.88510                                                                 
[22]	validation_0-auc:0.93724	validation_1-auc:0.88514                                                                 
[23]	validation_0-auc:0.93821	validation_1-auc:0.88504                                                                 
[24]	validation_0-auc:0.93888	validation_1-auc:0.88465                                                                 
[25]	validation_0-auc:0.93954	validation_1-auc:0.88449                                                                 
[26]	validation_0-auc:0.93999	validation_1-auc:0.88388                                                                 
[27]	validation_0-auc:0.94074	validation_1-auc:0.88424                                                                 
[28]	validation_0-auc:0.94174	validation_1-auc:0.88433                                                                 
[29]	validation_0-auc:0.94173	validation

[39]	validation_0-auc:0.93881	validation_1-auc:0.88663                                                                 
[40]	validation_0-auc:0.93881	validation_1-auc:0.88663                                                                 
[41]	validation_0-auc:0.93932	validation_1-auc:0.88647                                                                 
[42]	validation_0-auc:0.93968	validation_1-auc:0.88616                                                                 
[43]	validation_0-auc:0.93968	validation_1-auc:0.88616                                                                 
[44]	validation_0-auc:0.93999	validation_1-auc:0.88593                                                                 
[45]	validation_0-auc:0.93999	validation_1-auc:0.88593                                                                 
[46]	validation_0-auc:0.93999	validation_1-auc:0.88593                                                                 
[47]	validation_0-auc:0.94058	validation

In [18]:
# 평가용 함수
def  get_clf_eval(y_test, pred=None, pred_proba=None):
    confusion = confusion_matrix(y_test, pred)
    accuracy = accuracy_score(y_test, pred)
    precision = precision_score(y_test, pred)
    recall = recall_score(y_test, pred)
    f1 = f1_score(y_test, pred)
#     roc_auc = roc_auc_score(y_test, pred_proba)
    
    print('오차 행렬')
    print(confusion)
 
    print('정확도: {0:.4f}, 정밀도: {1:.4f}, \
    재현율: {2:.4f}, F1: {3:.4f}'.format(accuracy, precision, recall, f1))

In [19]:
xgbo = xgb.XGBClassifier(colsample_bytree=best['colsample_bytree'],
                         gamma=best['gamma'], max_depth=int(best['max_depth']),
                         learning_rate=best['learning_rate'],
                         min_child_weight=int(best['min_child_weight']), n_estimators=100,
                         random_state=109)
xgbo.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1,
              colsample_bytree=0.7430997391336394, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=1.158709508675603, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.19352376250081135, max_bin=256,
              max_cat_threshold=64, max_cat_to_onehot=4, max_delta_step=0,
              max_depth=5, max_leaves=0, min_child_weight=2, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=0,
              num_parallel_tree=1, predictor='auto', random_state=109, ...)

In [20]:
train_pred = xgbo.predict(X_train)
train_proba = xgbo.predict_proba(X_train)

test_pred = xgbo.predict(X_test)
test_proba = xgbo.predict_proba(X_test)

val_pred = xgbo.predict(X_val)
val_proba = xgbo.predict_proba(X_val)

In [21]:
get_clf_eval(y_train, train_pred, train_proba)

오차 행렬
[[1961  411]
 [ 289 2170]]
정확도: 0.8551, 정밀도: 0.8408,     재현율: 0.8825, F1: 0.8611


In [22]:
get_clf_eval(y_test, test_pred, test_proba)

오차 행렬
[[827 240]
 [203 878]]
정확도: 0.7938, 정밀도: 0.7853,     재현율: 0.8122, F1: 0.7985


In [23]:
get_clf_eval(y_val, val_pred, val_proba)

오차 행렬
[[633 185]
 [130 663]]
정확도: 0.8045, 정밀도: 0.7818,     재현율: 0.8361, F1: 0.8080


In [24]:
fi = pd.DataFrame(xgbo.feature_importances_, index=data.drop('Transported', axis=1).columns)
fi

,0
CryoSleep,0.332459
Cabin3,0.020072
Age,0.011779
VIP,0.008230
RoomService,0.039222
FoodCourt,0.027215
ShoppingMall,0.024658
Spa,0.046615
VRDeck,0.036004
Earth,0.117184


In [25]:
fi.to_csv('fi_3.csv')